In [107]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
%cd /content/drive/MyDrive/Saved-You-A-Click-CS224N - Copy/scraping

/content/drive/MyDrive/Saved-You-A-Click-CS224N - Copy/scraping


In [109]:
!pip install beautifulsoup4
!pip install google
!pip install newspaper3k

In [110]:
import csv
from difflib import SequenceMatcher
from googlesearch import search
import newspaper
from newspaper import Config
from newspaper import Article
import numpy as np
from os.path import exists
import pandas as pd

In [111]:
def removeLine(x):
    if x[0] =='|':
        return x[1:].strip()
    else:
        return x

In [112]:
def similar(a, b): 
    return SequenceMatcher(None, a, b).ratio()

In [113]:
df = pd.read_csv("data/scraped_posts/scraped_reddit_posts_25k.csv", usecols = ['url','title'])
list_of_urls= df['url'].values.tolist()
list_of_titles= df['title'].values.tolist()
print("Initial size of data set: " + str(len(df)))

# Filtering out results

# Keep only web archive or archive
# df2 = df[df['url'].str.contains("web.archive.org|https://archive.")] 
# archive links require extra processing. but decided this filter was too restrictive

# Remove youtube or reddit links
df2 = df[~df['url'].str.contains("https://www.youtube|https://i.redd.it|https://www.reddit.com/r/savedyouaclick")] 
print("After removing youtube or reddit links: " + str(len(df2)))

# Keep only entries that have teaser | answer format (removes ads top)
df3 = df2[df2['title'].str.contains("\\|")] #don't forget \\
print("After keeping only teaser | answer format: " + str(len(df3)))

Initial size of data set: 25213
After removing youtube or reddit links: 20626
After keeping only teaser | answer format: 17143


In [114]:
df3[['teaser', 'answer']] = df3['title'].str.split('|',n=1, expand=True)

USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = USER_AGENT
config.request_timeout = 10

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [115]:
def get_text_and_title(url):
    # modified from https://stackoverflow.com/questions/69711582/newspaper3k-export-to-csv-on-first-row-only
    article = newspaper.Article(url,config=config)
    article.download()
    article.parse()
    article_meta_data = article.meta_data
    text = article.text
    title = article.title
    return text, title

In [116]:
list_of_urls= df3['url'].values.tolist()
list_of_teasers= df3['teaser'].values.tolist()

index = 0
title_nomatch = 0
error_count = 0
list_of_texts = []
for url in list_of_urls:
    try:
        if '//archive.' in list_of_urls[index]:
            query = list_of_teasers[index]
            for j in search(query, tld="co.in", num=1, stop=1, pause=0):
                url = j
            text, title = get_text_and_title(url)
            if similar(title,list_of_teasers[index]) < 0.5:
                text = None
                print(index, 'title did not match',list_of_teasers[index],list_of_urls[index])
                title_nomatch += 1
            else:
                print(index)
        else:
            text, title = get_text_and_title(url)
            print(index)
    except:
        text = None 
        print(index, 'error',list_of_teasers[index],list_of_urls[index])
        error_count += 1
    index+=1
    list_of_texts.append(text)

print('Total data set: ', str(index))
print('Title did not match: ', str(title_nomatch))
print('Unknown error: ', str(error_count))



0
1
2 error Please don't saved you a click us, you guys. https://i.reddituploads.com/b9f24d1562664c5d810f3a629c451ad7?fit=max&h=1536&w=1536&s=a520aea7d1ab9c53839b64c001f0eb2a
3
4
5 error How This Woman Lost 44 Pounds Without *ANY* Exercise  http://archive.is/cfLqq
6 title did not match EXCLUSIVE: Tom Hanks Accused of Sexual Harassment by Former Coworker  http://archive.is/VNFUT
7
8
9
10
11
12 title did not match Britney Spears is pretty much the only celebrity whose coronavirus message hasn't annoyed everyone.  http://archive.is/Yo2nt
13
14
15
16
17
18 title did not match Sean Spicer Throws in the Towel  https://archive.is/SaNor
19
20
21
22
23
24
25
26
27
28
29
30 error This Is What Happens When You Fill A Car's Gas Tank With Coke  http://web.archive.org/save/http://jalopnik.com/here-is-what-happens-when-you-fill-a-cars-gas-tank-with-1821470264#_ga=2.255288334.70844216.1513555546-2064533995.1508124444
31
32
33 title did not match "Why No One in Hollywood Wants to Hire Jim Carrey"  http

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494 title did not match According to Tommy Chong, Here's Why CBD Doesn't Work  https://archive.is/URo4P
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508 title did not match Is Chrome really secretly stalking you across Google sites using per-install ID numbers? We reveal the truth  http://archive.is/370gm
1509
1510
1511 title did not match CNN Accidentally Cuts To Trump Too Early, What He Was Doing Will Make You Angry  http://archive.is/fJRpZ
1512
1513
1514
1515
1516
1517
1518
1519 title did not match Lady sees crying man forced to throw package in airport trash. What she digs out is heartbreaking  http://archive.is/3nd1F
1520
1521
1522
1523
1524
1525
1526 error Forbes 15 New Cars To Avoid  http://web.archive.org/save/http://www3.forbes.com/business/15-new-cars-to-avoid/16/
1527
1528 title did not match Pauly D shares how he dodged a bullet with ex Aubrey O’Day  http://archive.is/9KuM7?utm_source=sha

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


1805
1806 title did not match [Photos] He Took A Picture Of His Kids And Regretted It After Realizing What Was In The Background  https://archive.ph/auCPu
1807 title did not match BOY IS BULLIED AT BURGER KING AND THIS IS HOW OTHER CUSTOMERS REACT  https://archive.ph/Y1L0z
1808 title did not match Why You Might Want to Reconsider Going to Coachella After You Learn About Its Owner  http://archive.is/hBiRq
1809
1810
1811
1812
1813 error This Is How Beyonce Changed Her Nail Color Between Songs At Coachella  http://web.archive.org/save/http://www.papermag.com/beyonce-nails-color-coachella-2560645241.html
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832 error Did Samsung just release the iPhone 8?  http://web.archive.org/save/http://www.computerworld.com/video/76595/did-samsung-just-release-the-iphone-8
1833
1834
1835
1836
1837
1838
1839
1840 title did not match [Genius] "Today's Deal: Save 40% on Everything!"  http://archive.is/1PSiz
1841
1842


...

7698
7699
7700
7701
7702
7703
7704
7705
7706
7707
7708
7709
7710 title did not match East Street developer handed 'licence to kill'  http://archive.is/hW6yj
7711
7712
7713
7714
7715 error OWNER'S DOG LOOKS AT HIM WHILE HE FALLS ASLEEP, LATER FINDS OUT WHY  https://streamable.com/3x9te
7716
7717 error Why The Prison Population Is Climbing So Fast  https://streamable.com/iyonk
7718
7719 error 8 Things I Regret Buying for My First Apartment  https://streamable.com/ivdlc
7720
7721
7722
7723
7724
7725 title did not match During Excavation, Archeologists Saw Unusual Chest. What They Found Inside Made Everyone Hold Their Breath Away  http://archive.is/lD3nA
7726
7727
7728
7729
7730
7731
7732
7733
7734
7735
7736
7737 title did not match Judge Dismisses Grateful Teen’s Ticket, But Only On One Condition — And Mom Can’t Stop Smiling  http://archive.is/jiNsi
7738
7739
7740
7741 error Here's why Morgan has to die in The Walking Dead season 8  https://web.archive.org/save/http://www.digitalspy.com/t

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname MDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


8278
8279
8280
8281
8282 title did not match EVERYTHING TO KNOW ABOUT THE WESTERN RELEASE OF PERSONA 5 SCRAMBLE  http://archive.is/Uuypp
8283 title did not match Coronavirus: One Thing Could Tell Us Where the Next Outbreak Could Be (This Picture Is a Clue)  http://archive.today/HdWgS
8284
8285
8286 title did not match After Three Kids, Sight of New Pregnancy's Sonogram Make Father Faint  http://archive.is/zMmz8
8287 title did not match The Secret Nicknames Palace Security Use for Prince Harry and Meghan Markle  http://archive.is/NZEC9
8288
8289
8290 title did not match WhatsApp, a 322 euro fine for TIM users!  http://archive.is/PD6aP
8291
8292 error Why Is 'Gay Panic Defense' Still A Thing?  https://streamable.com/gpfgb
8293 error The Baby Orangutan Was Tied To A Tree Waiting For Help  http://web.archive.org/save/_embed/http://cyber-breeze.com/baby-orangutan-tied-tree-waiting-help/?utm_adid=37eb44&utm_source=Taboola&utm_medium=referral&utm_campaign=Tab-Villagers-Capture&utm_term=dailyk

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


12453
12454
12455
12456 error Physics, Not Genetics, Explains Why Flamingos Stand On One Leg  http://web.archive.org/save/https://www.forbes.com/sites/startswithabang/2019/10/10/physics-not-genetics-explains-why-flamingos-stand-on-one-leg/#56c31d8565de
12457
12458 error did iron man one set up why only tony stark could stop thanos?  http://web.archive.org/save/https://screenrant.com/iron-man-movie-tony-stark-doctor-strange-thanos/
12459
12460
12461
12462
12463 error This country just passed the U.S. as the world’s most competitive economy  https://web.archive.org/save/https://www.marketwatch.com/story/this-country-just-passed-the-us-as-the-worlds-most-competitive-economy-2019-10-08
12464
12465
12466 error The Controversial Scene That Took 'The Beverly Hillbillies' Off the Air  https://web.archive.org/save/https://www.history101.com/30-surprising-facts-about-the-beverly-hillbillies/?utm_source=faok&utm_medium=J_FB_H101_US_HillBillies_D1C1_1865_andios_1pv_con_open2_v1_0410&utm_content=fe

In [117]:
df3['article']= list_of_texts

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [118]:
df3['teaser'] = df3['teaser'].apply(lambda x: x.strip())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [119]:
df3 = df3[df3['title'].str.contains("\|")]

In [120]:
df3['answer'] = df3['answer'].apply(lambda x: x.strip())

In [121]:
df3 = df3.drop_duplicates(subset=['title'])

In [122]:
df3 = df3[df3['article']!='']

In [123]:
df3 = df3[df3['answer']!='']

In [124]:
df3['answer'] = df3['answer'].apply(removeLine)

In [125]:
df3.to_pickle('data_full_pandas.pkl')


In [126]:
dnp = df3.to_numpy()
np.save('data_full_numpy.npy', dnp)

In [128]:
df3

,title,url,teaser,answer,article
0,What the 'Someone Is Typing' Bubbles in Messag...,https://web.archive.org/web/20180721164300/htt...,What the 'Someone Is Typing' Bubbles in Messag...,It’s an indicator someone is typing,Your browser does not support HTML5 video tag....
1,What Getting Rid of the Electoral College woul...,https://web.archive.org/web/20190319232603/htt...,What Getting Rid of the Electoral College woul...,It would mean the person who gets the most vot...,"(CNN) On Monday night, in a CNN town hall in J..."
2,"Please don't saved you a click us, you guys.| ...",https://i.reddituploads.com/b9f24d1562664c5d81...,"Please don't saved you a click us, you guys.",Someone does. And reveals its Michael Crabtree...,None
3,IsConnor McGregor Hinting at a Possible Fight ...,https://web.archive.org/web/20170421055151/htt...,IsConnor McGregor Hinting at a Possible Fight ...,"No, Muhammad Ali is dead.",Original\n\nUFC superstar Conor McGregor was j...
4,"How Hillary Clinton Sill Can, And Should, Beco...",http://archive.is/Ag6M5,"How Hillary Clinton Sill Can, And Should, Beco...","Trump resigns, Pence resigns, Speaker Paul Rya...","Updated | Sure, it's been more than 340 days s..."
...,...,...,...,...,...
25204,What does the queen eat for breakfast? | The s...,http://web.archive.org/save/http://uk.business...,What does the queen eat for breakfast?,The souls of little children,None
25206,"Why Liberals Are Wrong About Trump | ""They're ...",http://web.archive.org/web/20170217052721/http...,Why Liberals Are Wrong About Trump,"""They're not"" + A recipe for raspberry scones.",Why Liberals Are Wrong About Trump\n\nWhy are ...
25207,WHY WE BROKE UP! | Two years ago. They're back...,https://streamable.com/wzxin,WHY WE BROKE UP!,Two years ago. They're back together and still...,None
25208,"Fox is Open to Rebooting Firefly, But Only Und...",http://web.archive.org/save/http://epicstream....,"Fox is Open to Rebooting Firefly, But Only Und...",Joss Whedon MUST Be Involved,None


In [127]:
df4 = df3[df3['title'].str.contains("\|")]